In [87]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from urllib.request import urlopen
from IPython.display import clear_output

In [88]:
# 요청 함수
def getSource(site) :
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers = header_info)
    # print(response.text)
    
    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [89]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(soup):
    
    # 데이터를 담은 딕셔너리
    data_dict = {
        '제목' : [],
        '평점' : [],
        '개봉날짜' : [],
        '예매율' : []
    }
    
    # 전체를 가지고 있는 태그를 가져온다
    a1 = soup.select_one('#mArticle > ul.list_movie')
    # print(a1)
    
    # 영화들을 가져온다
    a2 = a1.select('li')
    # print(a2)
    
    # 영화의 수만큼 반복한다
    for a3 in a2:
        # 영화 제목을 가져온다
        a4 = a3.select_one('div.wrap_movie > div > a')
        data1 = a4.text.strip()
        # print(data1)
        
        # 평점을 가져온다
        a5 = a3.select('div.wrap_movie > span.info_grade > a > span.wrap_grade.grade_netizen > span.num_grade')
        b1 = int(a5[0].text)
        b2 = int(a5[2].text)
        data2 = f'{b1}.{b2}'
        # print(data2)

        # 개봉일과 예매율을 가져온다
        a6 = a3.select_one('div.wrap_movie > span.info_state')
        b3 = a6.text.split('・')
        data3 = b3[0].strip()
        
        if len(b3) == 2:
            data4 = b3[1].strip()
        else:
            data4 = np.nan
        
        # 개봉날짜에서 '개봉' 문자열 제거
        data3 = data3.replace('개봉', '').strip()
        data3 = data3.replace('재', '').strip()
        
        # 예매율 제거, 퍼센트 제거
        if len(b3) == 2:
            data4 = data4.replace('예매율', '').strip()
            data4 = data4.replace('%', '').strip()
        
        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        
        # 영화 포스터
        a7 = a3.select_one('div.info_movie > span > img')
        # src 속성값을 가져온다
        src_attr = a7.attrs['src']
        # print(src_attr)
        
        if len(src_attr) > 0:
            # 포스터 이미지가 존재할 경우
            # 이미지 데이터를 읽어온다
            with urlopen('https:' + src_attr) as f1:
                img_data = f1.read()
                
            # 영화 제목의 특수문자를 제거한다
            filename = data1.replace(':', '-')
            filename = filename.replace('/', ' ')
            filename = filename.replace(';', '')
            
            # 저장
            with open(f'./poster/{filename}.jpg', 'wb') as f2: # 'wb'를 안 적으면 읽기전용으로 열려서 오류가 뜸
                f2.write(img_data)
        
        # 딕셔너리에 데이터를 담는다
        data_dict['제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개봉날짜'].append(data3)
        data_dict['예매율'].append(data4)
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('data2.csv') == False:
        # 파일이 없을 경우
        df1.to_csv('data2.csv', encoding='utf-8-sig', index=False) # encoding='utf-8-sig' : 엑셀로 열 때 한글이 안깨지도록
    else:
        df1.to_csv('data2.csv', encoding='utf-8-sig', index=False, header=False, mode='a') # mode='a' 기존의 것에 붙여줌

In [90]:
# 다음 페이지 존재 여부를 확인하는 함수

def getNext(soup):
    # 페이지 번호 중 a 태그를 가져온다
    a_tag = soup.select('#mArticle > div.paging_movie > span > a')
    # print(a_tag)
    
    # 페이지 번호 중 em 태그를 가져온다
    em_tag = soup.select_one('#mArticle > div.paging_movie > span > em')
    # print(em_tag)
    
    # a 태그 중 제일 마지막 태그의 숫자 값을 가져온다
    a1 = a_tag[-1].text.strip()
    # em 태그의 숫자값을 가져온다
    a2 = em_tag.text.strip()
    a2 = a2.replace('현재페이지', '')
    # print(a1)
    # print(a2)
    
    # 문자열이기 때문에 숫자로 바꾸기
    if int(a1) > int(a2):
        # 마지막 a 태그의 숫자가 em 태그의 숫자보다 더 큰 경우
        return True
    else:
        return False

In [91]:
page_name = 'released'
page = 1

while True:
    time.sleep(1)
    
    clear_output(wait=True)
    
    site = f'https://movie.daum.net/premovie/{page_name}?reservationOnly=N&sort=reservation&page={page}'
    
    print(f'{page_name} : {page} 수집중')
    
    soup = getSource(site)
    getData(soup)
    chk = getNext(soup)
    
    if chk == False:
        if page_name == 'released':
            page_name = 'scheduled'
            # 개봉 예정 페이지로 가면 다시 1페이지부터 시작해야 한다
            page = 1
        else:
            print('수집완료')
            break
    else:
        page += 1

scheduled : 6 수집중
수집완료


In [92]:
df10 = pd.read_csv('data2.csv')
df10

,제목,평점,개봉날짜,예매율
0,극장판 귀멸의 칼날: 무한열차편,5.5,21.01.27,29.8
1,소울,8.7,21.01.20,25.9
2,어니스트 씨프,6.8,21.02.03,5.7
3,해피 투게더,8.3,21.02.04,2.8
4,페어웰,8.3,21.02.04,1.7
...,...,...,...,...
142,꿈꾸는 고양이,10.0,21.06.04,NaN
143,달이 지는 밤,10.0,21.09.01,NaN
144,물방울을 그리는 남자,0.0,21.10.19,NaN
145,007 노 타임 투 다이,7.6,21.10,NaN


# 연습

In [38]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(soup):
    # 전체를 가지고 있는 태그를 가져온다
    a1 = soup.select_one('#mArticle > ul.list_movie')
    # print(a1)
    
    # 영화들을 가져온다
    a2 = a1.select('li')
    # print(a2)
    
    # 데이터를 담은 딕셔너리
    data_dict = {
        '영화제목' : [],
        '평점' : [],
        '개봉일' : []
    }
    
    # 영화의 수만큼 반복한다
    for a3 in a2:
        # 영화 제목을 가져온다
        a4 = a3.select_one('div.wrap_movie > div > a')
        data1 = a4.text.strip()
        # print(data1)
        
        # 평점을 가져온다
        a5 = a3.select('div.wrap_movie > span.info_grade > a > span.wrap_grade.grade_netizen > span.num_grade')
        b1 = int(a5[0].text)
        b2 = int(a5[2].text)
        data2 = f'{b1}.{b2}'
        # print(data2)

        # 개봉일과 예매율을 가져온다
        a6 = a3.select_one('div.wrap_movie > span.info_state')
        b3 = a6.text.split('・')
        data3 = b3[0].strip()
        # data4 = b3[1].strip()
        
        # 개봉날짜에서 '개봉' 문자열 제거
        data3 = data3.replace('개봉', '').strip()
        data3 = data3.replace('재', '').strip()
        
        # 예매율 제거, 퍼센트 제거
        # data4 = data4.replace('예매율', '').strip()
        # data4 = data4.replace('%', '').strip()
        
        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        
        # 딕셔너리에 데이터를 담는다
        data_dict['영화제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['개봉일'].append(data3)
        # data_dict['예매율'].append(data4)
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    display(df1)
    
#     if os.path.exists('data1.csv') == False:
#         # 파일이 없을 경우
#         df1.to_csv('data1.csv', encoding='utf-8-sig', index=False) # encoding='utf-8-sig' : 엑셀로 열 때 한글이 안깨지도록
#     else:
#         df1.to_csv('data1.csv', encoding='utf-8-sig', index=False, header=False, mode='a') # mode='a' 기존의 것에 붙여줌

In [39]:
# 다음 페이지 존재 여부 확인하는 함수
def getNextPage(soup, page_num):
    a1 = soup.select_one(f'#mArticle > div.paging_movie > span > a:nth-child({page_num})')
    if a1 == None:
        return 'None'

In [40]:
site = 'https://movie.daum.net/premovie/released'
page_path = '?reservationOnly=N&sort=reservation&page=%d'
page_num = 1

while True:
    sub_path = page_path%(page_num)  
    page_num += 1
    # 딜레이
    time.sleep(1)
    # 기존 출력된 것 청소하기
    clear_output(wait=True)
    # 페이지 요청
    soup = getSource(site + sub_path)
    # 데이터 수집
    getData(soup)
    if getNextPage(soup, page_num) == 'None':
        print('수집완료')
        break

,영화제목,평점,개봉일
0,마이 미씽 발렌타인,8.4,21.01.14
1,아이 엠 우먼,8.4,21.01.14
2,걸,8.8,21.01.07
3,미드나잇 버스,10.0,21.02.04
4,천사는 바이러스,8.8,21.01.06


수집완료
